Angelo Salton <gsalton4@hotmail.com>

# Setup

In [1]:
import pandas as pd
import numpy as np
import joblib

from ipywidgets import interact
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Pré-processamento dos dados

In [2]:
# salvar em formato Parquet
#X = pd.read_csv('../data/estaticos_market.csv')
#X.drop(columns='Unnamed: 0', inplace=True)

In [3]:
# amostra
#X = X.sample(5000)

In [4]:
# convert bool to int
#bool_cols = [*X.select_dtypes(include='bool').columns]
#X[bool_cols] = X[bool_cols].astype('int')

In [61]:
#X.to_parquet('../data/estaticos_market.parquet')
X = pd.read_parquet('../data/estaticos_market.parquet')

In [62]:
X.sample(5)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,vl_total_tancagem,vl_total_veiculos_antt,vl_total_veiculos_leves,vl_total_veiculos_pesados,fl_optante_simples,qt_art,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,vl_total_tancagem_grupo,vl_total_veiculos_antt_grupo,vl_potenc_cons_oleo_gas,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,de_indicador_telefone,fl_simples_irregular,vl_frota,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_st_suspensa,qt_socios_masculino,qt_socios_feminino,qt_socios_pep,qt_alteracao_socio_total,qt_alteracao_socio_90d,qt_alteracao_socio_180d,qt_alteracao_socio_365d,qt_socios_pj_ativos,qt_socios_pj_nulos,qt_socios_pj_baixados,qt_socios_pj_suspensos,qt_socios_pj_inaptos,vl_idade_media_socios_pj,vl_idade_maxima_socios_pj,vl_idade_minima_socios_pj,qt_coligados,qt_socios_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,idade_media_coligadas,idade_maxima_coligadas,idade_minima_coligadas,coligada_mais_nova_ativa,coligada_mais_antiga_ativa,idade_media_coligadas_ativas,coligada_mais_nova_baixada,coligada_mais_antiga_baixada,idade_media_coligadas_baixadas,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_ufs_coligados,qt_regioes_coligados,qt_ramos_coligados,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_funcionarios_coligados,qt_funcionarios_coligados_gp,media_funcionarios_coligados_gp,max_funcionarios_coligados_gp,min_funcionarios_coligados_gp,vl_folha_coligados,media_vl_folha_coligados,max_vl_folha_coligados,min_vl_folha_coligados,vl_folha_coligados_gp,media_vl_folha_coligados_gp,max_vl_folha_coligados_gp,min_vl_folha_coligados_gp,faturamento_est_coligados,media_faturamento_est_coligados,max_faturamento_est_coligados,min_faturamento_est_coligados,faturamento_est_coligados_gp,media_faturamento_est_coligados_gp,max_faturamento_est_coligados_gp,min_faturamento_est_coligados_gp,total_filiais_coligados,media_filiais_coligados,max_filiais_coligados,min_filiais_coligados,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,qt_coligadas,sum_faturamento_estimado_coligadas,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_ex_funcionarios,qt_funcionarios_grupo,percent_func_genero_masc,percent_func_genero_fem,idade_ate_18,idade_de_19_a_23,idade_de_24_a_28,idade_de_29_a_33,idade_de_34_a_38,idade_de_39_a_43,idade_de_44_a_48,idade_de_49_a_53,idade_de_54_a_58,idade_acima_de_58,grau_instrucao_macro_analfabeto,grau_instrucao_macro_escolaridade_fundamental,grau_instrucao_macro_escolaridade_media,grau_instrucao_macro_escolaridade_superior,grau_instrucao_macro_desconhecido,total,meses_ultima_contratacaco,qt_admitidos_12meses,qt_desligados_12meses,qt_desligados,qt_admitidos,media_meses_servicos_all,max_meses_servicos_all,min_meses_servicos_all,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
12969,2090841fe21a4a0d0aa010fda348394236de92724f9e62...,1,EMPRESARIO INDIVIDUAL,RO,OUTROS,COMERCIO E REPARACAO DE VEICULOS,COMERCIO,0.180822,<= 1,0,0,0,1,0,2018-08-23,0,0,1,NAO,COMERCIO E REPARA

In [7]:
#X = X.sample(10000).copy()

## Features categóricas

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

In [9]:
cat_features = [*X.select_dtypes(include='object').columns]
cat_features.remove('id')

cat_imputer = SimpleImputer(strategy='most_frequent', missing_values=None)
cat_encoder = OneHotEncoder(handle_unknown='error', sparse=True, drop='if_binary')

cat_transformer = Pipeline([
    ('cat_imp', cat_imputer),
    ('cat_enc', cat_encoder)
])

## Features numéricas

In [10]:
from sklearn.preprocessing import StandardScaler

In [11]:
# todas as features numéricas
num_features = [*X.select_dtypes(exclude='object').columns]

In [12]:
# features numéricas: 2 estratégias para dados faltantes, de acordo com a análise exploratória
nu1_features = [i for i in num_features if str.startswith(i, 'fl_')]
nu1_imputer = SimpleImputer(strategy='most_frequent')
nu1_scaler = StandardScaler()

nu1_transformer = Pipeline([
    ('nu1_imp', nu1_imputer),
    ('nu1_sca', nu1_scaler)
])

nu2_features = [i for i in num_features if str.startswith(i, 'qt_colig')]
nu2_imputer = SimpleImputer(strategy='constant', fill_value=0)

nu2_transformer = Pipeline([
    ('nu2_imp', nu2_imputer)
])

## Consolidado

In [13]:
from sklearn.compose import ColumnTransformer

In [14]:
data_transformer = ColumnTransformer([
    ('cat_tr', cat_transformer, cat_features),
    ('nu1_tr', nu1_transformer, nu1_features),
    ('nu2_tr', nu2_transformer, nu2_features)
], remainder='drop')

In [11]:
tmp = data_transformer.fit_transform(X)
tmp.shape

# Modelo

In [15]:
from sklearn.neighbors import NearestNeighbors

In [16]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)

In [17]:
pipe = Pipeline([
    ('data_tr', data_transformer),
    ('model', knn_model)
])

In [18]:
pipe.fit(X)

Pipeline(steps=[('data_tr',
                 ColumnTransformer(transformers=[('cat_tr',
                                                  Pipeline(steps=[('cat_imp',
                                                                   SimpleImputer(missing_values=None,
                                                                                 strategy='most_frequent')),
                                                                  ('cat_enc',
                                                                   OneHotEncoder(drop='if_binary'))]),
                                                  ['de_natureza_juridica',
                                                   'sg_uf',
                                                   'natureza_juridica_macro',
                                                   'de_ramo', 'setor',
                                                   'idade_emp_cat',
                                                   'dt_situacao', 'fl_rm',
               

In [19]:
pipe.named_steps['model'].kneighbors(\
    data_transformer.transform(X.sample(10)), 3
)

(array([[0.00000000e+00, 1.44379596e-01, 1.44379596e-01],
        [0.00000000e+00, 4.08276086e-02, 8.16552173e-02],
        [0.00000000e+00, 4.51339541e-02, 1.35401862e-01],
        [3.33066907e-16, 1.37750159e-01, 1.56889360e-01],
        [0.00000000e+00, 1.24401212e-01, 1.65868283e-01],
        [0.00000000e+00, 3.18038551e-02, 3.49356485e-02],
        [0.00000000e+00, 1.61779428e-01, 1.61779428e-01],
        [0.00000000e+00, 4.04448569e-02, 4.04448569e-02],
        [4.44089210e-16, 4.28314804e-02, 4.28314804e-02],
        [0.00000000e+00, 8.08897138e-02, 8.08897138e-02]]),
 array([[8176, 1238, 1800],
        [4203, 9499, 3192],
        [8139, 5236, 6448],
        [3727, 8041, 5340],
        [5453, 3951, 2148],
        [1567, 7712, 5725],
        [2556, 4738, 6694],
        [9557, 1359, 4155],
        [5907,  607, 2254],
        [1722, 1078, 3644]], dtype=int64))

# Testes

In [22]:
sample_ids = X.sample(10)['id']

In [27]:
sample_ids

387894    2dc83104c051a670b241eedeb744e5a1e81048f1dbed83...
453843    a01a9c724d3b226b7a4a830635cbf7f3752544803b8d18...
392968    eaa826bad2eaccfb5977f35a847940e81e47243f8b290b...
452887    7782381b4dc9537406200a93d25a14190fe87888b89a16...
34165     aec70665392513c6da0afd91f521101945e3bad0eafffc...
410264    8e1c7173691da0cef2322bb15f434f0fbd445250448939...
393654    7e6596bd648c98e7ea37ed5f30c7b3f902dd9fc0135a4b...
60700     c4da39334979298140bacaa3a9b59d4a097495bce02b12...
272182    83f0c91b480a952334db31292c1187249008b4ad1b7a11...
358888    2c1e594d59e705d239e9d38231729d34186ec6f57a17dd...
Name: id, dtype: object

In [56]:
def recommendations(ids: pd.Series, n: int):
    '''
    Extrai recomendações do modelo.
    '''
    data = X.loc[ids.index, :]
    
    distances, indexes = pipe.named_steps['model'].kneighbors(\
        data_transformer.transform(data), n
    )

    ids = X.iloc[indexes[:, 0].flatten().tolist(), 0]
    
    return distances, ids

In [57]:
ds, ids = recommendations(sample_ids, 4)

# Pickling

In [85]:
# exporta o modelo treinado
joblib.dump(pipe, '../model.joblib')

['model.joblib']